In [1]:
def vec_from_by(ax,fro=[0,0],delta=[1,1],scale=1,col='black',label='',ls='-'):
    ax.text(fro[0]+delta[0]/2,fro[1]+delta[1]/3,label,color=col,fontsize=20)
    return ax.arrow(fro[0], fro[1], delta[0], delta[1], head_width=0.1*scale, head_length=0.15*scale, color=col,length_includes_head=True,ls=ls)

In [1]:
def line_from_by(ax,fro=[0,0],delta=[1,1],scale=1,col='black',label='',ls='-'):
    ax.text(fro[0]+delta[0]/2*scale,fro[1]+delta[1]/3*scale,label,color=col,fontsize=20)
    return ax.arrow(fro[0], fro[1], delta[0]*scale, delta[1]*scale, head_width=0, head_length=0, color=col,length_includes_head=True,ls=ls)

In [2]:
def define_element(geo,ids):
    block = geo.loc[geo['Pkt'] < 0] #emtpy
    for i in ids:
        block = block.append(geo.loc[geo['Pkt'] == i])
    return block

In [5]:
def plot_blocks(blocks):
    fig, ax = plt.subplots()
    for i,block in enumerate(blocks):
        for k in range(len(block)):
            ax.plot([block.iloc[k]['x / m']],[block.iloc[k]['y / m']],marker='o',color='black')
            ax.text(block.iloc[k]['x / m'],block.iloc[k]['y / m']+0.2, str(int(block.iloc[k]['Pkt'])))
        block = block.append(block.iloc[0])#to close the loop
        ax.plot(block['x / m'], block['y / m'],label='Element %i' %(i+1))#,color='black')
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$y$ / m')
    ax.legend()
    fig.tight_layout()

In [5]:
def auto_plot_blocks(geometry):
    block1 = define_element(geometry,[1,2,3,4])
    block2 = define_element(geometry,[4,3,6,5])
    block3 = define_element(geometry,[5,6,7,8])
    block4 = define_element(geometry,[8,7,9])
    plot_blocks([block1,block2,block3,block4])

In [ ]:
def block_area(block):
    if len(block) == 3:
        A = np.abs(block.iloc[0]['x / m'] * (block.iloc[1]['y / m'] - block.iloc[2]['y / m']) + 
                   block.iloc[1]['x / m']* (block.iloc[2]['y / m'] - block.iloc[0]['y / m']) + 
                   block.iloc[2]['x / m'] * (block.iloc[0]['y / m'] - block.iloc[1]['y / m']))/2
    elif len(block) == 4:
        A = np.abs((block.iloc[2]['x / m'] - block.iloc[0]['x / m']) * (block.iloc[3]['y / m'] - block.iloc[1]['y / m']) + 
                   (block.iloc[3]['x / m'] - block.iloc[1]['x / m']) * (block.iloc[0]['y / m'] - block.iloc[2]['y / m'])) / 2
    else:
        raise SystemExit('Element neither triangle nor quadrilateral')
    return A

In [6]:
def length(geo,ids):
    a = geo.loc[geo['Pkt']==ids[0]]
    b = geo.loc[geo['Pkt']==ids[1]]
    L = np.sqrt((a.iloc[0]['x / m'] - b.iloc[0]['x / m'])**2 + (a.iloc[0]['y / m'] - b.iloc[0]['y / m'])**2)
    return L

In [7]:
def gravity_vector(data,nr):
    return np.array([0,-data.iloc[nr-1]['G / kN/m']])

In [ ]:
def cohesion(c,geo,from_id,to_id):
    L = length(geo,[from_id,to_id])
    from_node = geo.loc[geo['Pkt']==from_id]
    to_node = geo.loc[geo['Pkt']==to_id]
    vec = np.array([to_node.iloc[0]['x / m'] - from_node.iloc[0]['x / m'], to_node.iloc[0]['y / m'] - from_node.iloc[0]['y / m']])
    vec /= np.linalg.norm(vec)
    vec *= L * c
    return vec

In [8]:
def line_of_Q(geo,R,from_id,to_id):
    from_node = geo.loc[geo['Pkt']==from_id]
    to_node = geo.loc[geo['Pkt']==to_id]
    vec = R.dot(np.array([to_node.iloc[0]['x / m'] - from_node.iloc[0]['x / m'], to_node.iloc[0]['y / m'] - from_node.iloc[0]['y / m']]))
    vec /= np.linalg.norm(vec)
    return vec

In [ ]:
def get_P_T(eta, *args):
    eta=eta[0]
    geometry = args[0]
    gamma = args[1]
    c=args[2]
    phi=args[3]
    plotfig=args[4]
    #mobilised values
    c /= eta
    phi = np.arctan(np.tan(np.deg2rad(phi))/eta)
    #Achtung: Nummerierung im Umlaufsinn!
    block1 = define_element(geometry,[1,2,3,4])
    block2 = define_element(geometry,[4,3,6,5])
    block3 = define_element(geometry,[5,6,7,8])
    block4 = define_element(geometry,[8,7,9])
    #
    data = {'A / m²': np.array([block_area(block1),block_area(block2),block_area(block3),block_area(block4)])} #in m
    data['G / kN/m'] = data['A / m²']*gamma
    blockdata = pd.DataFrame(data)
    #
    beta = np.deg2rad(90) - phi
    Rp = np.array([[np.cos(beta),-np.sin(beta)],[np.sin(beta),np.cos(beta)]])
    Rn = np.array([[np.cos(-beta),-np.sin(-beta)],[np.sin(-beta),np.cos(-beta)]])
    #
    G_4 = gravity_vector(blockdata,4)
    C_98 = cohesion(c,geometry,9,8)
    C_78 = cohesion(c,geometry,7,8)
    e_Q98 = line_of_Q(geometry,Rp,9,8)
    e_Q78 = line_of_Q(geometry,Rn,7,8)
    b4 = -(G_4 + C_98 + C_78)
    A4 = np.array([e_Q78,e_Q98]).T
    f4 = np.linalg.solve(A4,b4)
    Q_78 = e_Q78 * f4[0]
    Q_98 = e_Q98 * f4[1]
    #
    G_3 = gravity_vector(blockdata,3)
    C_87 = - C_78
    Q_87 = - Q_78
    #
    C_85 = cohesion(c,geometry,8,5)
    C_65 = cohesion(c,geometry,6,5)
    e_Q85 = line_of_Q(geometry,Rp,8,5)
    e_Q65 = line_of_Q(geometry,Rn,6,5)
    b3 = -(G_3 + C_87 + C_85 + C_65 + Q_87)
    A3 = np.array([e_Q85,e_Q65]).T
    f3 = np.linalg.solve(A3,b3)
    Q_85 = e_Q85 * f3[0]
    Q_65 = e_Q65 * f3[1]
    #
    G_2 = gravity_vector(blockdata,2)
    C_56 = - C_65
    Q_56 = - Q_65
    C_54 = cohesion(c,geometry,5,4)
    C_34 = cohesion(c,geometry,3,4)
    e_Q54 = line_of_Q(geometry,Rp,5,4)
    e_Q34 = line_of_Q(geometry,Rn,3,4)
    b2 = -(G_2 + C_56 + C_54 + C_34 + Q_56)
    A2 = np.array([e_Q54,e_Q34]).T
    f2 = np.linalg.solve(A2,b2)
    Q_54 = e_Q54 * f2[0]
    Q_34 = e_Q34 * f2[1]
    #
    G_1 = gravity_vector(blockdata,1)
    C_43 = - C_34
    Q_43 = - Q_34
    C_41 = cohesion(c,geometry,4,1)
    e_Q41 = line_of_Q(geometry,Rp,4,1)
    e_PT = np.array([0,-1])
    b1 = -(G_1 + C_43 + C_41 + Q_43)
    A1 = np.array([e_Q41,e_PT]).T
    f1 = np.linalg.solve(A1,b1)
    Q_41 = e_Q41 * f1[0]
    P_T = e_PT * f1[1]
    #
    if plotfig:
        scale = np.linalg.norm(G_2)
        fig, ax = plt.subplots(figsize=(12,12))
        vec_from_by(ax,delta=G_1,label='$G_1$',scale=scale)
        vec_from_by(ax,fro=G_1,delta=G_2,label='$G_2$',scale=scale)
        vec_from_by(ax,fro=G_1+G_2,delta=G_3,label='$G_3$',scale=scale)
        vec_from_by(ax,fro=G_1+G_2+G_3,delta=G_4,label='$G_4$',scale=scale)
        #
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4,delta=Q_98,label='$Q_{98}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98,delta=C_98,label='$C_{98}$',scale=scale,col='blue')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98,delta=Q_85,label='$Q_{85}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85,delta=C_85,label='$C_{85}$',scale=scale,col='blue')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85,delta=Q_54,label='$Q_{54}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54,delta=C_54,label='$C_{54}$',scale=scale,col='blue')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54+C_54,delta=Q_41,label='$Q_{41}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54+C_54+Q_41,delta=C_41,label='$C_{41}$',scale=scale,col='blue')
        #
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54+C_54+Q_41+C_41,delta=P_T,label='$P_\\mathrm{T} = %.2f kN/m$' %(np.linalg.norm(P_T)),scale=scale,col='red')
        ax.set(adjustable='box', aspect='equal')
        plt.axis('off');
        plt.show();
    obj_value = np.linalg.norm(P_T)
    return obj_value